In [ ]:
import pandas as pd
import arcpy

# Corrected file path with raw string
file_path = r'C:\Users\yyang295\Desktop\Sunrise_Sunset_Intervals_2023_LA.xlsx'
df = pd.read_excel(file_path)

# Convert time to datetime format for manipulation
df['DateTime'] = pd.to_datetime(df['Time'], format='%I:%M %p')

# Generate end times by adding 10 minutes to start times
df['EndDateTime'] = df['DateTime'] + pd.Timedelta(minutes=10)

# Convert back to string format for use in the script
df['StartTime'] = df['DateTime'].dt.strftime('%m/%d/%Y %I:%M:%S %p')
df['EndTime'] = df['EndDateTime'].dt.strftime('%m/%d/%Y %I:%M:%S %p')

# Prepare time intervals from the dataframe as a list of tuples
time_intervals = df[['StartTime', 'EndTime']].apply(tuple, axis=1).tolist()

def Model4(time_intervals):
    arcpy.env.overwriteOutput = False
    arcpy.CheckOutExtension("3D")

    exbuild_Layer3DToFeatureClas = "exbuild_Layer3DToFeatureClas"
    exfree_Layer3DToFeatureClass = "exfree_Layer3DToFeatureClass"
    extree_Layer3DToFeatureClass = "extree_Layer3DToFeatureClass"
    ImageServer = "https://elevation3d.arcgis.com/arcgis/services/WorldElevation3D/Terrain3D/ImageServer"

    for interval in time_intervals:
        start_time, end_time = interval

        jan1_filename = fr"C:\Users\yyang295\Documents\ArcGIS\Packages\yifan1030_0029e5\p30\myproject.gdb\jan1_{start_time.replace('/', '').replace(':', '').replace(' ', '')}"
        jan12_filename = fr"C:\Users\yyang295\Documents\ArcGIS\Packages\yifan1030_0029e5\p30\myproject.gdb\jan12_{start_time.replace('/', '').replace(':', '').replace(' ', '')}"

        arcpy.ddd.SunShadowFrequency(in_features=[exbuild_Layer3DToFeatureClas, exfree_Layer3DToFeatureClass, extree_Layer3DToFeatureClass], ground=ImageServer, out_raster=jan1_filename, start_time=start_time, end_time=end_time, time_interval="10 Minutes")
        jan1 = arcpy.Raster(jan1_filename)

        arcpy.ddd.SunShadowFrequency(in_features=[exbuild_Layer3DToFeatureClas, exfree_Layer3DToFeatureClass], ground=ImageServer, out_raster=jan12_filename, start_time=start_time, end_time=end_time)
        jan12 = arcpy.Raster(jan12_filename)

if __name__ == '__main__':
    with arcpy.EnvManager(scratchWorkspace=r"C:\Users\yyang295\Documents\ArcGIS\Packages\yifan1030_0029e5\p30\myproject.gdb", workspace=r"C:\Users\yyang295\Documents\ArcGIS\Packages\yifan1030_0029e5\p30\myproject.gdb"):
        Model4(time_intervals)